In [2]:
import requests
import pandas as pd

In [5]:
URL_API_STATION_AVAILABLE =  "https://velib-metropole-opendata.smovengo.cloud/opendata/Velib_Metropole/station_status.json"
URL_API_STATION_INFOS = "https://velib-metropole-opendata.smovengo.cloud/opendata/Velib_Metropole/station_information.json"

Data_Set_Station_Available = requests.get(URL_API_STATION_AVAILABLE).json()
Data_Set_Station_Infos = requests.get(URL_API_STATION_INFOS).json()

df_status =  pd.json_normalize(Data_Set_Station_Available['data']['stations'])
df_infos = pd.json_normalize(Data_Set_Station_Infos['data']['stations'])


df_status = df_status[['station_id', 'num_bikes_available', 'num_bikes_available_types', 'num_docks_available',
      'is_returning', 'is_renting','is_installed', 'last_reported', 'stationCode']]

df_infos = df_infos[['station_id', 'name', 'lat', 'lon', 'capacity']]

df_status['last_reported'] = df_status['last_reported'].apply(lambda x: pd.to_datetime(x, unit='s'))
df_status['num_available_ebikes'] = df_status['num_bikes_available_types'].apply(lambda x: x[1]['ebike'] if x and len(x)>1 else 0)
df_status['num_available_mechanical_bikes'] = df_status['num_bikes_available_types'].apply(lambda x: x[0]['mechanical'] if x and len(x)>1 else 0)
df_status = df_status.drop(columns=['num_bikes_available_types'])
##print(df_status)
##print(df_infos)

df_merged = df_status.merge(df_infos, on ='station_id', how = 'left')
print(df_merged.columns)

df_merged.to_csv('velib_dataset.csv', index=False)

Index(['station_id', 'num_bikes_available', 'num_docks_available',
       'is_returning', 'is_renting', 'is_installed', 'last_reported',
       'stationCode', 'num_available_ebikes', 'num_available_mechanical_bikes',
       'name', 'lat', 'lon', 'capacity'],
      dtype='object')
